<a href="https://colab.research.google.com/github/tharoosha/Learner_Aware_AI/blob/model_testing/Copy_of_SocratiQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!git clone https://github.com/NUS-IDS/eacl23_soqg.git


Cloning into 'eacl23_soqg'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 254 (delta 31), reused 114 (delta 30), pack-reused 136 (from 1)
Receiving objects: 100% (254/254), 40.70 MiB | 28.74 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# load csv file in to dataframe
df_test = pd.read_csv('/content/eacl23_soqg/data/mturk_annotations/mturk_test.csv')

In [ ]:
df_test.head()

,Unnamed: 0,thought,question,label
0,0,They made the choice to have sex with their pa...,Why do you come to the presumption that taboo ...,reasons_evidence
1,1,They're exploiting the technologically illiter...,And what happens when all the same sources rep...,implication_consequences
2,2,They're denying special rights. It's a special...,What if we just allow all bathrooms to be used...,implication_consequences
3,3,1. Better implementation of the report functio...,Are you implying Reddit should hire an admin w...,implication_consequences
4,4,What portion of land doesn’t have significance...,Why do you feel that indigenous tribes need to...,reasons_evidence


In [ ]:
df_train = pd.read_csv("/content/eacl23_soqg/data/mturk_annotations/mturk_train_valid.csv")
df_train.head()

,Unnamed: 0,thought,question,label
0,0,I’m a vegetarian and would hate to eat either....,"So, I assume then you also think that eating h...",assumptions
1,1,"By our current standards, a child is just as l...",How is that different from a real sword in any...,clarity
2,2,"Christianity, as I understand it, teaches that...",Why are you yelling?,others
3,3,There are people in this country who work way ...,What exactly are these barriers preventing peo...,clarity
4,4,I have. I was paid minimum wage at a job. The ...,What if your job creates only 10 dollars an hour?,implication_consequences


In [ ]:
df_train.shape

(2535, 4)

In [ ]:
pip install tabulate

In [ ]:
import pandas as pd
import os
import numpy as np
import torch
from torch import cuda
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os
from datetime import datetime
import sys
from tabulate import tabulate
from transformers import T5Tokenizer, T5ForConditionalGeneration
import logging

In [ ]:
device = "cuda:0" if cuda.is_available() else 'cpu'

In [ ]:
train_df_chunk_I = pd.read_csv("/content/eacl23_soqg/data/soqg_dataset/train_chunk_I.csv", index_col=0)
train_df_chunk_II = pd.read_csv("/content/eacl23_soqg/data/soqg_dataset/train_chunk_II.csv", index_col=0)
train_df_chunk_III = pd.read_csv("/content/eacl23_soqg/data/soqg_dataset/train_chunk_III.csv", index_col=0)

In [ ]:
train_df_chunk_I.head()

,input,target
0,alternate_viewpoints_perspectives: A parallel ...,What about nations who have nothing?
1,alternate_viewpoints_perspectives: It would be...,"If not, what about this is cringe exactly?"
2,alternate_viewpoints_perspectives: I do not un...,What about public surveillance cameras?
3,alternate_viewpoints_perspectives: There is a ...,How about allowing some students to go straigh...
4,alternate_viewpoints_perspectives: You conside...,What else do you imagine is necessary to be co...


In [ ]:
train_df_chunk_II.head()

,input,target
28194,clarity: My rationale is quite simple so Im wo...,Are women born with Y chromosomes men?
28195,clarity: Discussions about confrontations with...,How would that come up in a discussion regardi...
28196,clarity: They're also not capable of that thou...,Are we also predators?
28197,clarity: Trump knowing full well the election ...,Is that not possible?
28198,clarity: I did address it. Be pedantic all you...,What is the purpose of a punishment?


In [ ]:
print(train_df_chunk_I.shape,train_df_chunk_II.shape,train_df_chunk_III.shape)

(28194, 2) (28194, 2) (28194, 2)


In [ ]:
train_df = pd.concat([train_df_chunk_I, train_df_chunk_II, train_df_chunk_III], axis = 0)

valid_df = pd.read_csv("/content/eacl23_soqg/data/soqg_dataset/valid.csv", index_col=0)
test_df = pd.read_csv("/content/eacl23_soqg/data/soqg_dataset/test.csv", index_col=0)

train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)


In [ ]:
logging.info(len(train_df))
logging.info(len(valid_df))
logging.info(len(test_df))

In [ ]:
model_params = {
        "TOKENIZER_PATH": "t5-large",
        "MODEL_PATH": "t5-large",
        "TRAIN_BATCH_SIZE": 4,
        "VALID_BATCH_SIZE": 4,
        "TRAIN_EPOCHS": 20,
        "LEARNING_RATE": 5e-5,
        "MAX_SOURCE_TEXT_LENGTH": 400,
        "MAX_TARGET_TEXT_LENGTH": 80,
        "SEED": 0,
    }

In [ ]:
class T5_Dataset(Dataset):

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
         ):

        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        return len(self.target_text)

    def __getitem__(self, index):
        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])
        source_text = " ".join(source_text.split())
        target_text = "[Question] " + " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer, training_logger):
  logging.info("The number of batches to load / training epoch is " + str(len(loader)))
  model.train()
  total_loss = 0

  for _, data in enumerate(loader, 0):
      y = data["target_ids"].to(device, dtype=torch.long)
      y_ids = y[:, :-1].contiguous()
      lm_labels = y[:, 1:].clone().detach()
      lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
      ids = data["source_ids"].to(device, dtype=torch.long)
      mask = data["source_mask"].to(device, dtype=torch.long)

      outputs = model(
          input_ids=ids,
          attention_mask=mask,
          decoder_input_ids=y_ids,
          labels=lm_labels,
      )
      loss = outputs[0]
      total_loss += loss.item()

      if _ % 1000 == 0:
          training_logger.append([str(epoch), str(_), str(loss)])
          logging.info(tabulate(training_logger))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  logging.info(f"Epoch Total Train Loss: {total_loss}")
  return total_loss


In [ ]:
def validate(epoch, tokenizer, model, device, loader, scheduler, valid_logger):

    logging.info("The number of batches to load / validation epoch is " + str(len(loader)))
    model.eval()
    total_loss = 0

    with torch.no_grad():

        for _, data in enumerate(loader, 0):
            y = data["target_ids"].to(device, dtype=torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data["source_ids"].to(device, dtype=torch.long)
            mask = data["source_mask"].to(device, dtype=torch.long)

            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                decoder_input_ids=y_ids,
                labels=lm_labels,
            )
            loss = outputs[0]
            total_loss += loss.item()

            if _ % 1000 == 0:
                valid_logger.append([str(epoch), str(_), str(loss)])
                logging.info(tabulate(valid_logger))

    logging.info(f"Epoch Total Valid Loss: {total_loss}")
    scheduler.step(total_loss)
    return total_loss

In [ ]:
def predict(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    logging.info("Validation Predictions: \n")
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask,
              max_length=150,
              do_sample=True,
              top_k = 5,
              top_p = 0.6,
              repetition_penalty=1.2,
              no_repeat_ngram_size = 3
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%1000==0:
                logging.info(f'Completed validation step {_}')

            predictions.extend(preds)
            actuals.extend(target)

    total_num_pred = len(predictions)
    for i, prediction in enumerate(predictions):
        if i%int(0.1*total_num_pred) == 0:
            logging.info(f"Pred@{i}:{prediction}")
    return predictions, actuals


In [ ]:
def start_training(train_dataset, val_dataset, test_dataset, source_text, target_text, model_params, output_dir="/content/T5_train_Outputs"):
    best_epoch = -999999
    torch.manual_seed(model_params["SEED"])
    np.random.seed(model_params["SEED"])
    torch.backends.cudnn.deterministic = True
    print(f"""[Model]: Loading {model_params["MODEL_PATH"]}...\n""")
    tokenizer = T5Tokenizer.from_pretrained(model_params["TOKENIZER_PATH"])
    tokenizer.add_tokens("[Question]")
    print(f"[Question] token added {tokenizer.added_tokens_encoder}")
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL_PATH"])
    model = model.to(device)
    logging.info(f"[Data]: Reading data...\n")
    train_dataset = train_dataset[[source_text, target_text]]
    val_dataset = val_dataset[[source_text, target_text]]
    test_dataset = test_dataset[[source_text, target_text]]
    print(f"TRAIN Dataset: {train_dataset.shape}")
    print(f"VALID Dataset: {val_dataset.shape}\n")
    print(f"TEST Dataset: {test_dataset.shape}\n")
    training_set = T5_Dataset(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = T5_Dataset(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    test_set = T5_Dataset(
        test_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }
    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }
    test_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)
    test_loader = DataLoader(test_set, **test_params)
    optimizer = torch.optim.AdamW(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2, threshold=0.01)
    print(f"[Initiating Fine Tuning]...\n")
    lowest_valid_loss = 999999

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        try:

            training_logger = [["Epoch: ", "Step: ", "Loss: "]]
            valid_logger = [["Epoch: ", "Step: ", "Loss: "]]

            epoch_train_loss = train(epoch, tokenizer, model, device, training_loader, optimizer, training_logger)
            epoch_valid_loss = validate(epoch, tokenizer, model, device, val_loader, scheduler, valid_logger)

            if epoch_valid_loss < lowest_valid_loss:
                print(f"Lowest Valid Loss reached @ epoch {epoch}")
                lowest_valid_loss = epoch_valid_loss
                predictions, actuals = predict(epoch, tokenizer, model, device, test_loader)
                print(f"[Saving Model @ Epoch {epoch}]...\n")
                best_epoch = epoch
                path = os.path.join(output_dir, "model_files_T5_large_conditional")
                model.save_pretrained(path)
                tokenizer.save_pretrained(path)
                test_df["Generated_Question"] = predictions
                test_df["Actual_Question"] = actuals
                test_df.to_csv(os.path.join(output_dir, "t5_prediction_test_df.csv"))

        except Exception as e:
            print(e)

    print(f"Training completed")
    print(f"Best Model @{best_epoch}")

In [ ]:
start_training(
    train_dataset=train_df,
    val_dataset=valid_df,
    test_dataset=test_df,
    source_text="input",
    target_text="target",
    model_params=model_params,
    output_dir="/content/Train_Outputs",)

[Model]: Loading t5-large...



spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[Question] token added {'<pad>': 0, '</s>': 1, '<unk>': 2, '<extra_id_99>': 32000, '<extra_id_98>': 32001, '<extra_id_97>': 32002, '<extra_id_96>': 32003, '<extra_id_95>': 32004, '<extra_id_94>': 32005, '<extra_id_93>': 32006, '<extra_id_92>': 32007, '<extra_id_91>': 32008, '<extra_id_90>': 32009, '<extra_id_89>': 32010, '<extra_id_88>': 32011, '<extra_id_87>': 32012, '<extra_id_86>': 32013, '<extra_id_85>': 32014, '<extra_id_84>': 32015, '<extra_id_83>': 32016, '<extra_id_82>': 32017, '<extra_id_81>': 32018, '<extra_id_80>': 32019, '<extra_id_79>': 32020, '<extra_id_78>': 32021, '<extra_id_77>': 32022, '<extra_id_76>': 32023, '<extra_id_75>': 32024, '<extra_id_74>': 32025, '<extra_id_73>': 32026, '<extra_id_72>': 32027, '<extra_id_71>': 32028, '<extra_id_70>': 32029, '<extra_id_69>': 32030, '<extra_id_68>': 32031, '<extra_id_67>': 32032, '<extra_id_66>': 32033, '<extra_id_65>': 32034, '<extra_id_64>': 32035, '<extra_id_63>': 32036, '<extra_id_62>': 32037, '<extra_id_61>': 32038, '<ext

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

TRAIN Dataset: (84582, 2)
VALID Dataset: (10573, 2)

TEST Dataset: (10573, 2)

[Initiating Fine Tuning]...

